This notebook was created to obtain pseudo-ground truth images for the study. The ground truth images were generated using K-means over Planet-NICFI imagery, then water bodies and building footprint products available on GEE were used to refine it.

Note: this notebook was used on Google Colab, thus it might be necessary to install some packages in the environment if you want to run it locally. Also the directories need to be changed.


# 0) Install and import packages

In [ ]:
import os

import ee
import geemap
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt

from shapely.geometry import Point
from google.colab import files

In [ ]:
Map = geemap.Map() # initialize GEE and geemap

# 1) Data upload and transformation


In [ ]:
uploads = files.upload()

In [ ]:
asm_sites = gpd.read_file('/content/visible_asm.geojson')

In [ ]:
# convert dataset to GeoSeries
asm_series = gpd.GeoSeries(asm_sites.geometry)

# set WGS 84 as CRS to display everything with geemap
asm_series = asm_series.to_crs(4326)

# create buffers, distance is in degrees, roughly 1 km
asm_buffer = asm_series.buffer(distance=0.008)

asm_boxes = asm_buffer.envelope

In [ ]:
# check if the boxes have been correctly created
fig, ax1 = plt.subplots()
asm_boxes.boundary.plot(ax=ax1, color='red')

In [ ]:
# convert the rectangles to ee.Geometry objects
ee_boxes = []
for geom in asm_boxes:
    coords = list(geom.exterior.coords)
    bbox_coords = [coords[0], coords[2]]  # get 1st and 3rd coord
    rectangle = ee.Geometry.Rectangle(bbox_coords)
    ee_boxes.append(rectangle)


# create a single geometry that combines all the bounding boxes
combined_geometry = ee.FeatureCollection(ee_boxes)

# 2) Get satellite imagery and products to obtain pseudo-ground truth


In [ ]:
# get Planet-NiCFI median composite of the first half of 2023
nicfi_median_2023 = ee.ImageCollection('projects/planet-nicfi/assets/basemaps/africa') \
    .filterDate('2023-01-01','2023-6-30') \
    .median()

# clip imagery with bounding boxes
nicfi_images = nicfi_median_2023.clipToCollection(combined_geometry)

In [ ]:
# get the JRC Global Surface Water Mapping Layers product
water = ee.Image("JRC/GSW1_4/GlobalSurfaceWater")
water = water.clipToCollection(combined_geometry)

# resample to match Planet-NICFI pixel size
water = water.resample('bilinear').reproject(crs='EPSG:4326', scale=4.77)

water_vis = {
    'bands': ['max_extent'],
    'min': 0.0,
    'max': 1.0
}

In [ ]:
# Get World Settlement Footprint product
urban = ee.Image('DLR/WSF/WSF2015/v1')
urban = urban.clipToCollection(combined_geometry)
urban_vis = {
    'min': 0,
    'max': 255
}

In [ ]:
# get Open Buildings product
buildings = ee.FeatureCollection('GOOGLE/Research/open-buildings/v3/polygons') \
                    .filter('confidence >= 0.75')

In [ ]:
# diplay everything on an interactive map
vis = {'bands':['R','G','B'],
       'min':64,
       'max':5454,
       'gamma':1.8}

fc = combined_geometry.style(
    color='red'
)

Map = geemap.Map(center=[0, 0], zoom=2)
Map.addLayer(fc, {}, 'Boxes')
Map.addLayer(nicfi_images, vis, 'Planet-NICFI images')
Map.addLayer(water, water_vis, 'Water bodies')
Map.addLayer(urban, urban_vis, 'Urban areas')
Map.addLayer(buildings, {'color': '00FF00'}, 'Buildigs')
Map

# 3) Image segmentation via K-means clustering


In [ ]:
def Kmeans(input, clusters_number, study_area, scale, num_pixels=1000):
    """
    Perform K-means clustering on input imagery.

    Parameters:
    - input: ee.Image, the input image to be clustered.
    - clusters_number: int, the number of clusters to form.
    - study_area: ee.Geometry, the region over which to perform the clustering.
    - scale: float, the spatial resolution in meters.
    - num_pixels: int, number of pixels to sample for clustering.

    Returns:
    - ee.Image with the clustered classification.
    """
    # make a sample from the input image for training
    training = input.sample(
        region=study_area,
        scale=scale,
        numPixels=num_pixels
    )

    # create clusterer
    clusterer = ee.Clusterer.wekaKMeans(
        nClusters=clusters_number,
        init=1,
        seed=10).train(training)

    # apply clustering to the input image
    classification = input.cluster(clusterer).select(0).rename('unsupervised_class')

    return classification

Hereby we select 4 clusters as we expect 4 different land use types in the study area: ASM, forest, urban  areas, water bodies

In [ ]:
# define parameters for the clustering
n_clusters = 4
scale = 4.77
num_pixels = 5000

# run clustering function
Kmeans_segment = Kmeans(nicfi_with_ndvi,
            n_clusters,
            combined_geometry.geometry(),
            scale,
            num_pixels)

In the case of K-means, need to check in the map below which cluster represents the ASM sites and modify the two remap lists. Note: this part is hard-coded, and the cluster related to the ASM sites can change at each run

In [ ]:
Map.addLayer(Kmeans_segment.select('unsupervised_class').randomVisualizer(),
             {},
             'K-means segmentation',
             True)

In [ ]:
# reclassify the segmented training images
from_list = [0, 1, 2, 3]
to_list = [0, 1, 0, 0] # cluster 1 is the cluster of the ASM sites

Kmeans_reclass = Kmeans_segment.remap(from_list, to_list, defaultValue=0,
                                        bandName='unsupervised_class')

In [ ]:
# add the reclassified raster on the map
Map.addLayer(Kmeans_reclass, None, 'K-means reclassified')
Map

In [ ]:
# create binary mask to exclude building footporint
buildings_mask = ee.Image().paint(buildings, 1).unmask(0).Not()
buildings_mask = buildings_mask.clipToCollection(combined_geometry)

# create binary mask to exclude water bodies
water_mask = water.select('max_extent').gt(0).Not() # invert the mask

# create binary mask to exclude urban areas
urban_mask = urban.select('settlement').gt(0).eq(1)
urban_mask = urban_mask.where(urban_mask, 0)
urban_mask = urban_mask.unmask(1).clipToCollection(combined_geometry) # fill the Null pixels with 1

Map.addLayer(water_mask, {}, 'Water Mask')
Map.addLayer(urban_mask, {}, 'Urban Mask')
Map.addLayer(buildings_mask, {}, 'Buildings Mask')

# combine masks
combined_mask = water_mask.And(urban_mask).And(buildings_mask)
Map.addLayer(combined_mask, {}, 'Combined Mask')

# apply combined mask to the ground truth layer
gt_mask = Kmeans_reclass.where(combined_mask.eq(0), 0)
Map.addLayer(gt_mask, {}, 'Masked GT')

Map

# 4) Export results from GEE

Hereby the images are exported from GEE to Google Drive as .tif files. A loop for each subset is needed (trainining images, training ground truth, testing images, testing ground truth).

In [ ]:
# convert boxes feature collections to list
boxes_list = combined_geometry.toList(combined_geometry.size())

In [ ]:
ee.Feature(boxes_list.get(0)).geometry().crs

In [ ]:
# loop over the list of boxes and export Planet images
for i in range(boxes_list.size().getInfo()):
    # get i-th feature
    feature = ee.Feature(boxes_list.get(i))

    # get the feature geometry
    geometry = feature.geometry()

    # define export params
    train_export_params = {
        'image': nicfi_images,
        'description': 'nicfi_' + str(i),
        'folder': 'thesis_planet_images',
        'scale': 4.77,
        'region': geometry,
        'maxPixels': 1e13
    }

    # export image to drive
    task = ee.batch.Export.image.toDrive(**train_export_params)
    task.start()

In [ ]:
# loop over the list of boxes and export ground truth images
for i in range(boxes_list.size().getInfo()):
    # get i-th feature
    feature = ee.Feature(boxes_list.get(i))

    # get the feature geometry
    geometry = feature.geometry()

    # define export params
    train_export_params = {
        'image': gt_mask,
        'description': 'nicfi_gt_' + str(i),
        'folder': 'thesis_Kmeans_gt',
        'scale': 4.77,
        'region': geometry,
        'maxPixels': 1e13
    }

    # export image to drive
    task = ee.batch.Export.image.toDrive(**train_export_params)
    task.start()